### Librerias utilizadas

In [37]:
import pandas as pd
import re
import ast


In [5]:
df1 = pd.read_csv('movies_dataset.csv')

C:\Users\Luis Fernando Martin\AppData\Local\Temp\ipykernel_9748\538979729.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('movies_dataset.csv')


In [7]:
# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [50]:
#df1

#### Primer paso es desanidar las columnas anidadas 

In [12]:
# Buscamos si está, y si está busca la clave 'id' y captura todo el texto que sigue hasta la próxima comilla simple, excepto la ,
busqueda1 = r"'id': ([^,]*)"
df1['coll_id'] = df1['belongs_to_collection'].str.extract(busqueda1, flags=re.IGNORECASE)

 #Si 'coll_id' es nulo (NaN), es que no contiene los valores extraídos de id de la colección.
busqueda2 = r"'name': '([^']*)"
df1['coll_name'] = df1['belongs_to_collection'].str.extract(busqueda2, flags=re.IGNORECASE)

 #Creamos una nueva columna 'coll_name' que contiene los valores extraídos del nombre de la colección.
#df1.head(5)

In [14]:
#Desanide las columna "production_companies"
Muest_1 = r"'id': ([^}]*)"
df1['production_company_id'] = df1['production_companies'].str.extract(Muest_1, flags=re.IGNORECASE)
Muest_2= r"'name': '([^']*)"
df1['production_company_name'] = df1['production_companies'].str.extract(Muest_2, flags=re.IGNORECASE)
#df1.head(5)

### Rellenar valores nulos de los columnas revenue, budget

In [17]:
# Cuento la cantidad de valores nulos en las columnas 'revenue' y 'budget'
nulos_revenue = df1['revenue'].isnull().sum()
nulos_budget = df1['budget'].isnull().sum()

# Mostrar la cantidad de valores nulos en cada columna
print(f"Valores nulos en 'revenue': {nulos_revenue}")
print(f"Valores nulos en 'budget': {nulos_budget}")


Valores nulos en 'revenue': 0
Valores nulos en 'budget': 0


In [18]:
# Rellenar los valores nulos en la columna 'revenue' con 0
df1['revenue'] = df1['revenue'].fillna(0)
#df1

### Verifico los valores NaN de la columna release_date y los elimino

In [21]:
# Contar valores NaN en la columna 'release_date'
nan_count = df1['release_date'].isna().sum()
nan_count

0

In [20]:
# Elimine donde 'release_date' tiene valores NaN
df1 = df1.dropna(subset=['release_date'])

### Estraer el año de la fecha de estreno de la pelicula 

In [24]:
# Convertir la columna 'release_date' al formato 'AAAA-mm-dd' si contiene fechas
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Crear la columna 'release_year' extrayendo el año de la columna 'release_date' y convertir a entero
df1['release_year'] = pd.to_datetime(df1['release_date'], errors='coerce').dt.year.astype('Int64')

# Mostrar el DataFrame para verificar los cambios
#df1


C:\Users\Luis Fernando Martin\AppData\Local\Temp\ipykernel_9748\2275649560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')
C:\Users\Luis Fernando Martin\AppData\Local\Temp\ipykernel_9748\2275649560.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['release_year'] = pd.to_datetime(df1['release_date'], errors='coerce').dt.year.astype('Int64')


### Dividiendo las dos columnas revenue y budget y despues obtemos la columna return con el toltal de la divicion 

In [27]:
# Convertir las columnas 'revenue' y 'budget' a tipo numérico, forzando errores a NaN
df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')
df1['budget'] = pd.to_numeric(df1['budget'], errors='coerce')

# Crear la columna 'return' calculando revenue / budget
df1['return'] = df1.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)
#df1

C:\Users\Luis Fernando Martin\AppData\Local\Temp\ipykernel_9748\2051867996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')
C:\Users\Luis Fernando Martin\AppData\Local\Temp\ipykernel_9748\2051867996.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['budget'] = pd.to_numeric(df1['budget'], errors='coerce')
C:\Users\Luis Fernando Martin\AppData\Local\Temp\ipykernel_9748\2051867996.py:6: SettingWithCopyWarning: 
A value is trying to be se

### Elimar las columnas que nos sugirieron a eliminar 

In [32]:
# Verifico si las columnas a eliminar existen en el DataFrame
columns_to_remove = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
existing_columns = [col for col in columns_to_remove if col in df1.columns]

# Elimino las columnas que existen en el DataFrame
df1.drop(columns=existing_columns, inplace=True)
#df1

In [35]:
# Verificar si las columnas existen y eliminarlas si es así
columns_to_drop = ['belongs_to_collection', 'production_companies']

# Eliminar las columnas si existen
df1 = df1.drop(columns=[col for col in columns_to_drop if col in df1.columns], errors='ignore')

# Mostrar el DataFrame para confirmar que las columnas han sido eliminadas
#df1

### Desanide la columna genres

In [46]:
# # Función para extraer los géneros sin corchetes
def extract_genres(genres_list):
#     # Verificar si genres_list es una lista y no es vacía
     if isinstance(genres_list, list) and genres_list:
         genres = [genre['name'] for genre in genres_list if 'name' in genre]
         return ', '.join(genres)  # Unir los géneros en una cadena separada por comas
     return ''  # Devolver una cadena vacía si no hay géneros

# # Suponiendo que 'df1' es tu DataFrame y 'genres' es la columna que quieres desanidar
df1['genres'] = df1['genres'].apply(lambda x: extract_genres(ast.literal_eval(x)) if pd.notna(x) and x != 'NaN' else '')

# # Mostrar los primeros registros para verificar el resultado
# #print(df1['genres'].head())
#df1

In [48]:
# Contar valores duplicados en la columna 'id'
duplicados = df1['id'].duplicated(keep=False).sum()

# Contar valores NaN en la columna 'id'
nan_count = df1['id'].isna().sum()

print(f"Cantidad de valores duplicados en la columna 'id': {duplicados}")
print(f"Cantidad de valores NaN en la columna 'id': {nan_count}")

Cantidad de valores duplicados en la columna 'id': 0
Cantidad de valores NaN en la columna 'id': 0


In [43]:
# Eliminar valores duplicados en la columna 'id', manteniendo solo la primera ocurrencia
#df1 = df1.drop_duplicates(subset='id', keep='first')

# Verificar el resultado
#print(df1.head())

### Guarde el archivo ya modificado

In [49]:
# Guardar el DataFrame en un archivo CSV
df1.to_csv('movies.modificado.csv', index=False)
